# Problem Statement 
The pet training centre has been experiencing high call and visit volumes by pet owners who seek advice from staff when they need help in handling their pets (cats and dogs in particular) as the current chatbot was not giving relevant information to the customers. Such behaviour results in a waste of manpower resources as the staff had to spend a huge part of their time speaking with pet owners at the centre or over the phone. If the chatbot was able to classify the enquiries accurately and provide relevant information, the bulk of the enquiries could be handled automatically.

As a data analyst at the training centre, I was tasked to improve the existing classifier model used by the chatbot, or design a new model with increased accuracy.

## Data Cleaning and Processing

In [93]:
# import packages
import pandas as pd
import string
import numpy as np
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# widen columns
pd.options.display.max_colwidth = 400

In [94]:
# read data
cat_data = pd.read_csv('../data/cat_raw.csv')
dog_data = pd.read_csv('../data/dog_raw.csv')

In [95]:
# to confirm number of rows of data
cat_data.shape

(5997, 78)

In [96]:
# to confirm number of rows of data
dog_data.shape

(5999, 81)

In [97]:
# to check no. of submissions with text removed by moderator. 
# cat_data has 2336 rows out of 5997 (39%) data with text removed.
# this is a huge proportion of the data set. as such, we keep the submissions, but analyse the 'title' instead for datapoints without 'selftext'

cat_data[cat_data['selftext']=='[removed]'].count()

Unnamed: 0                2336
all_awardings             2336
allow_live_comments       2336
author                    2336
author_flair_css_class       0
                          ... 
thumbnail_height             0
thumbnail_width              0
event_end                    0
event_is_live                0
event_start                  0
Length: 78, dtype: int64

In [98]:
# to check for null entries
# the 71 null entries out of 5997 (1.2%) for 'selftext' can be represented by contents from 'title'
cat_data['selftext'].isnull().value_counts()

False    5926
True       71
Name: selftext, dtype: int64

In [99]:
# to check no. of submissions with text removed by moderator. 
# dog_data only has 47 rows out of 5999 (0.8%) data with text removed.
dog_data[dog_data['selftext']=='[removed]'].count()

Unnamed: 0                       47
all_awardings                    47
allow_live_comments              47
author                           47
author_flair_css_class            0
                                 ..
media_metadata                    0
author_flair_background_color     0
author_flair_text_color           0
author_cakeday                    0
poll_data                         0
Length: 81, dtype: int64

In [100]:
# to check for null entries
# the 2005 null entries out of 5999 (33.4%) for 'selftext' can be represented by contents from 'title'
# many of these post are videos or images. we will look at the title content to explore the content further
dog_data['selftext'].isnull().value_counts()

False    3994
True     2005
Name: selftext, dtype: int64

Since the proportion of datapoints with text removed by moderator for both categories is very large, we will keep those for both cat and dog data sets to ensure data points were not lost. The title could be used to identify what were the topic(s) of the submissions. For the remaining submissions, we will combine the contents under 'title' and 'selftext'.

In [101]:
# check data type for title and selftexts
cat_data[['title','selftext']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5997 entries, 0 to 5996
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     5997 non-null   object
 1   selftext  5926 non-null   object
dtypes: object(2)
memory usage: 93.8+ KB


In [102]:
# drop rows based on repeated input for 'title' column and keep only the first occurence 
cat_data = cat_data.drop_duplicates(subset=['title'], keep='first')
dog_data = dog_data.drop_duplicates(subset=['title'], keep='first')
cat_data.reset_index(drop=True, inplace=True)
dog_data.reset_index(drop=True, inplace=True)

In [103]:
# 65 rows of data removed
cat_data.shape

(5932, 78)

In [104]:
# 104 rows of data removed
dog_data.shape

(5895, 81)

#### Combine "title" and "selftext" Contents

In [105]:
# replace null values with "."
cat_data['selftext'].fillna(value = '.', inplace = True)
dog_data['selftext'].fillna(value = '.', inplace = True)

In [106]:
#cat data
# combine both title and selftext columns with "|". if selftext content is [removed] or '.', return "title"
cat_data["full_content_cat"] = None
for key, value in cat_data.iterrows():
    if cat_data['selftext'][key] != '[removed]':
        cat_data["full_content_cat"][key] = str(cat_data["title"][key]) + " | " + str(cat_data["selftext"][key])
    else:
        cat_data["full_content_cat"][key] = cat_data["title"][key]

C:\Users\gyinh\AppData\Local\Temp\ipykernel_14660\1871414572.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_data["full_content_cat"][key] = str(cat_data["title"][key]) + " | " + str(cat_data["selftext"][key])
C:\Users\gyinh\AppData\Local\Temp\ipykernel_14660\1871414572.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_data["full_content_cat"][key] = cat_data["title"][key]


In [107]:
#dog data
# combine both title and selftext columns with "|". if selftext content is [removed] or '.', return "title"
dog_data["full_content_dog"] = None
for key, value in dog_data.iterrows():
    if dog_data['selftext'][key] != '[removed]':
        dog_data["full_content_dog"][key] = str(dog_data["title"][key]) + " | " + str(dog_data["selftext"][key])
    else:
        dog_data["full_content_dog"][key] = dog_data["title"][key]

C:\Users\gyinh\AppData\Local\Temp\ipykernel_14660\234375725.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dog_data["full_content_dog"][key] = str(dog_data["title"][key]) + " | " + str(dog_data["selftext"][key])
C:\Users\gyinh\AppData\Local\Temp\ipykernel_14660\234375725.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dog_data["full_content_dog"][key] = dog_data["title"][key]


In [108]:
# to check if output is appearing as intended
cat_data['full_content_cat'][633]

'Can you use Feliway before neutering?'

In [109]:
# function to remove numbers from text
def remove_numbers(text):
    text_nonum = re.sub(r'[\d+]', "", text)
    return text_nonum

# remove numbers from text for cat and dog data
cat_data['full_content_cat'] = cat_data['full_content_cat'].apply(lambda x: remove_numbers(x))
dog_data['full_content_dog'] = dog_data['full_content_dog'].apply(lambda x: remove_numbers(x))

In [110]:
# function to remove URL
def remove_url(text):
    text_nourl = re.sub(r'http\S+', "", text)
    return text_nourl

# remove url from text for cat and dog data
cat_data['full_content_cat'] = cat_data['full_content_cat'].apply(lambda x: remove_url(x))
dog_data['full_content_dog'] = dog_data['full_content_dog'].apply(lambda x: remove_url(x))

In [111]:
dog_data['full_content_dog']

0       Urgent, need info asap | Can anybody identify what this could possibly be? When I first adopted my dog about - months ago (Staffordshire Terrier) we noticed that she had this little branch off on her paw pad and thought nothing of it because it looked like just a minor paw abnormality at first. We noticed today after being out and playing that it has started to look like this and we aren’t sur...
1       Behavior problems with corgi | Hello all, I've been having serious behavioral problems with my M pembroke welsh corgi and he has been targeting my M Cairn terrier. We have had both dogs since they were - months old and though my corgi has always been "grumpy" he has never been aggressive with the cairn terrier. My cairn terrier is a very calm old man that is incontinent but never bothered any ...
2                                                                                                                                                                                       

#### Remove Punctuations

In [112]:
# function to keep the text and remove punctuations found in string.punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

# clean up for cat and dog datasets
cat_data['full_content_cat'].map(str)
cat_data['full_content_cat_punc'] = cat_data['full_content_cat'].apply(lambda x: remove_punct(x))

dog_data['full_content_dog'].map(str)
dog_data['full_content_dog_punc'] = dog_data['full_content_dog'].apply(lambda x: remove_punct(x))

In [113]:
cat_data[['full_content_cat','full_content_cat_punc']].tail()

,full_content_cat,full_content_cat_punc
5927,"Should I take in a cat? | Hello everyone, one of my brother's friend can't take care of another cat and wish to give it to someone who can. And I really want a cat but the question I have is : should I take the cat in even tho I still live with my parents and they have a garden and a backyard where the cat can wander around and stuff the thing is, I wait to have a contract where I work to take...",Should I take in a cat Hello everyone one of my brothers friend cant take care of another cat and wish to give it to someone who can And I really want a cat but the question I have is should I take the cat in even tho I still live with my parents and they have a garden and a backyard where the cat can wander around and stuff the thing is I wait to have a contract where I work to take a flat ...
5928,"Got a new kitten, how can I help her get along with my dog?",Got a new kitten how can I help her get along with my dog
5929,Teenager Troubles,Teenager Troubles
5930,"Food for sterilized cats | My kitten was recently sterilized, she is months old. Is it worth transferring her to food for sterilized cats while she is still a kitten, or is it worth continuing to feed her with food for kittens? Now I'm feeding her ""blue buffalo healthy growth natural kitten food""",Food for sterilized cats My kitten was recently sterilized she is months old Is it worth transferring her to food for sterilized cats while she is still a kitten or is it worth continuing to feed her with food for kittens Now Im feeding her blue buffalo healthy growth natural kitten food
5931,"Cat acting out and anxious after getting new kitten. | So I got a new kitten about a month ago and introduced him to my cat ( year old so still young). I did the correct introduction by separating them, swapping scents, eating near eachother etc. and everything was going really well. They started playing all the time, licking eachother, and seemed to be best buds. Now suddenly my older cat is ...",Cat acting out and anxious after getting new kitten So I got a new kitten about a month ago and introduced him to my cat year old so still young I did the correct introduction by separating them swapping scents eating near eachother etc and everything was going really well They started playing all the time licking eachother and seemed to be best buds Now suddenly my older cat is hissing all ...


In [114]:
dog_data[['full_content_dog','full_content_dog_punc']].tail()

,full_content_dog,full_content_dog_punc
5890,"Dog has lost all her stomach and chest hair. | My year old French Bulldog has lost all her hair since three months ago. I don’t know if it’s due to stress (we got a new puppy exactly three months ago, she had all her hair back then) or due to a possible medical illness due to age. I took her to the vet and all tests came out normal, I asked for an X-Ray to look out for tumors. Dr. who knows a...",Dog has lost all her stomach and chest hair My year old French Bulldog has lost all her hair since three months ago I don’t know if it’s due to stress we got a new puppy exactly three months ago she had all her hair back then or due to a possible medical illness due to age I took her to the vet and all tests came out normal I asked for an XRay to look out for tumors Dr who knows about ultras...
5891,"My toy poodle’s fur has been thinning and falling out in one spot on her back, what should I do? | .",My toy poodle’s fur has been thinning and falling out in one spot on her back what should I do
5892,"My dog wants to be let out every night, in the middle of the night (- am). | He’s around years old and he’s wonderfully house trained, like never has an accident ever. This is a new (annoying) habit that has started within the past couple months. We always let him out before we go to bed (around pm) and his last meal is usually around pm. His water is typically empty towards the evening, so...",My dog wants to be let out every night in the middle of the night am He’s around years old and he’s wonderfully house trained like never has an accident ever This is a new annoying habit that has started within the past couple months We always let him out before we go to bed around pm and his last meal is usually around pm His water is typically empty towards the evening so we just don’t ...
5893,"Dogs getting spayed… I have some questions. | Both my dogs are getting spayed on Wednesday (//). One dog is about years old and the other is almost years old. I’m nervous for the procedure, but I know that it is routine and better for them in the long term. I planned their spay to occur when I’ll be off from work for a week to be able to care for them. \n\n year old = black mouth cur \n year...",Dogs getting spayed… I have some questions Both my dogs are getting spayed on Wednesday One dog is about years old and the other is almost years old I’m nervous for the procedure but I know that it is routine and better for them in the long term I planned their spay to occur when I’ll be off from work for a week to be able to care for them \n\n year old black mouth cur \n year old black ...
5894,"Pup chews everything on walks | My month old pup tries to chews everything on walks. Pinecones, branches, rocks, grass, etc. Any tips on how to improve it? I’ve googled a million things and they say one thing could be boredom. That’s the whole point of a walk? Get some physical activity? But he won’t even make it far on a walk and just lays down and chews grass :( I’ve tried doing food puzzle...",Pup chews everything on walks My month old pup tries to chews everything on walks Pinecones branches rocks grass etc Any tips on how to improve it I’ve googled a million things and they say one thing could be boredom That’s the whole point of a walk Get some physical activity But he won’t even make it far on a walk and just lays down and chews grass I’ve tried doing food puzzles and play in...


#### Tokenisation

In [115]:
# matches any character that is neither alphanumeric nor underscore
# add a + just in case there are 2 or more spaces between certain words
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

#tokenise cat and dog data sets
cat_data['full_content_cat_token'] = cat_data['full_content_cat_punc'].apply(lambda x: tokenize(x.lower()))
dog_data['full_content_dog_token'] = dog_data['full_content_dog_punc'].apply(lambda x: tokenize(x.lower()))

In [116]:
cat_data[['full_content_cat_punc','full_content_cat_token']].tail()

,full_content_cat_punc,full_content_cat_token
5927,Should I take in a cat Hello everyone one of my brothers friend cant take care of another cat and wish to give it to someone who can And I really want a cat but the question I have is should I take the cat in even tho I still live with my parents and they have a garden and a backyard where the cat can wander around and stuff the thing is I wait to have a contract where I work to take a flat ...,"[should, i, take, in, a, cat, hello, everyone, one, of, my, brothers, friend, cant, take, care, of, another, cat, and, wish, to, give, it, to, someone, who, can, and, i, really, want, a, cat, but, the, question, i, have, is, should, i, take, the, cat, in, even, tho, i, still, live, with, my, parents, and, they, have, a, garden, and, a, backyard, where, the, cat, can, wander, around, and, stuff..."
5928,Got a new kitten how can I help her get along with my dog,"[got, a, new, kitten, how, can, i, help, her, get, along, with, my, dog]"
5929,Teenager Troubles,"[teenager, troubles]"
5930,Food for sterilized cats My kitten was recently sterilized she is months old Is it worth transferring her to food for sterilized cats while she is still a kitten or is it worth continuing to feed her with food for kittens Now Im feeding her blue buffalo healthy growth natural kitten food,"[food, for, sterilized, cats, my, kitten, was, recently, sterilized, she, is, months, old, is, it, worth, transferring, her, to, food, for, sterilized, cats, while, she, is, still, a, kitten, or, is, it, worth, continuing, to, feed, her, with, food, for, kittens, now, im, feeding, her, blue, buffalo, healthy, growth, natural, kitten, food]"
5931,Cat acting out and anxious after getting new kitten So I got a new kitten about a month ago and introduced him to my cat year old so still young I did the correct introduction by separating them swapping scents eating near eachother etc and everything was going really well They started playing all the time licking eachother and seemed to be best buds Now suddenly my older cat is hissing all ...,"[cat, acting, out, and, anxious, after, getting, new, kitten, so, i, got, a, new, kitten, about, a, month, ago, and, introduced, him, to, my, cat, year, old, so, still, young, i, did, the, correct, introduction, by, separating, them, swapping, scents, eating, near, eachother, etc, and, everything, was, going, really, well, they, started, playing, all, the, time, licking, eachother, and, seemed..."


In [117]:
dog_data[['full_content_dog_punc','full_content_dog_token']].tail()

,full_content_dog_punc,full_content_dog_token
5890,Dog has lost all her stomach and chest hair My year old French Bulldog has lost all her hair since three months ago I don’t know if it’s due to stress we got a new puppy exactly three months ago she had all her hair back then or due to a possible medical illness due to age I took her to the vet and all tests came out normal I asked for an XRay to look out for tumors Dr who knows about ultras...,"[dog, has, lost, all, her, stomach, and, chest, hair, my, year, old, french, bulldog, has, lost, all, her, hair, since, three, months, ago, i, don, t, know, if, it, s, due, to, stress, we, got, a, new, puppy, exactly, three, months, ago, she, had, all, her, hair, back, then, or, due, to, a, possible, medical, illness, due, to, age, i, took, her, to, the, vet, and, all, tests, came, out, normal..."
5891,My toy poodle’s fur has been thinning and falling out in one spot on her back what should I do,"[my, toy, poodle, s, fur, has, been, thinning, and, falling, out, in, one, spot, on, her, back, what, should, i, do, ]"
5892,My dog wants to be let out every night in the middle of the night am He’s around years old and he’s wonderfully house trained like never has an accident ever This is a new annoying habit that has started within the past couple months We always let him out before we go to bed around pm and his last meal is usually around pm His water is typically empty towards the evening so we just don’t ...,"[my, dog, wants, to, be, let, out, every, night, in, the, middle, of, the, night, am, he, s, around, years, old, and, he, s, wonderfully, house, trained, like, never, has, an, accident, ever, this, is, a, new, annoying, habit, that, has, started, within, the, past, couple, months, we, always, let, him, out, before, we, go, to, bed, around, pm, and, his, last, meal, is, usually, around, pm, his..."
5893,Dogs getting spayed… I have some questions Both my dogs are getting spayed on Wednesday One dog is about years old and the other is almost years old I’m nervous for the procedure but I know that it is routine and better for them in the long term I planned their spay to occur when I’ll be off from work for a week to be able to care for them \n\n year old black mouth cur \n year old black ...,"[dogs, getting, spayed, i, have, some, questions, both, my, dogs, are, getting, spayed, on, wednesday, one, dog, is, about, years, old, and, the, other, is, almost, years, old, i, m, nervous, for, the, procedure, but, i, know, that, it, is, routine, and, better, for, them, in, the, long, term, i, planned, their, spay, to, occur, when, i, ll, be, off, from, work, for, a, week, to, be, able, to,..."
5894,Pup chews everything on walks My month old pup tries to chews everything on walks Pinecones branches rocks grass etc Any tips on how to improve it I’ve googled a million things and they say one thing could be boredom That’s the whole point of a walk Get some physical activity But he won’t even make it far on a walk and just lays down and chews grass I’ve tried doing food puzzles and play in...,"[pup, chews, everything, on, walks, my, month, old, pup, tries, to, chews, everything, on, walks, pinecones, branches, rocks, grass, etc, any, tips, on, how, to, improve, it, i, ve, googled, a, million, things, and, they, say, one, thing, could, be, boredom, that, s, the, whole, point, of, a, walk, get, some, physical, activity, but, he, won, t, even, make, it, far, on, a, walk, and, just, lay..."


#### Remove Stopwords

In [118]:
# call stopwords for english from NLTK
stoplist = nltk.corpus.stopwords.words('english')


# retain and store word in "text" if it is not found in "stopword"
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stoplist]
    return text

#remove stopwords for cat and dog data sets
cat_data['full_content_cat_nostop'] = cat_data['full_content_cat_token'].apply(lambda x: remove_stopwords(x))
dog_data['full_content_dog_nostop'] = dog_data['full_content_dog_token'].apply(lambda x: remove_stopwords(x))

In [119]:
cat_data[['full_content_cat_token','full_content_cat_nostop']].tail()

,full_content_cat_token,full_content_cat_nostop
5927,"[should, i, take, in, a, cat, hello, everyone, one, of, my, brothers, friend, cant, take, care, of, another, cat, and, wish, to, give, it, to, someone, who, can, and, i, really, want, a, cat, but, the, question, i, have, is, should, i, take, the, cat, in, even, tho, i, still, live, with, my, parents, and, they, have, a, garden, and, a, backyard, where, the, cat, can, wander, around, and, stuff...","[take, cat, hello, everyone, one, brothers, friend, cant, take, care, another, cat, wish, give, someone, really, want, cat, question, take, cat, even, tho, still, live, parents, garden, backyard, cat, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wondering, bother, cat, change, environment, like, tough, adapt, new, style, life, wait, till, get, flat, ok..."
5928,"[got, a, new, kitten, how, can, i, help, her, get, along, with, my, dog]","[got, new, kitten, help, get, along, dog]"
5929,"[teenager, troubles]","[teenager, troubles]"
5930,"[food, for, sterilized, cats, my, kitten, was, recently, sterilized, she, is, months, old, is, it, worth, transferring, her, to, food, for, sterilized, cats, while, she, is, still, a, kitten, or, is, it, worth, continuing, to, feed, her, with, food, for, kittens, now, im, feeding, her, blue, buffalo, healthy, growth, natural, kitten, food]","[food, sterilized, cats, kitten, recently, sterilized, months, old, worth, transferring, food, sterilized, cats, still, kitten, worth, continuing, feed, food, kittens, im, feeding, blue, buffalo, healthy, growth, natural, kitten, food]"
5931,"[cat, acting, out, and, anxious, after, getting, new, kitten, so, i, got, a, new, kitten, about, a, month, ago, and, introduced, him, to, my, cat, year, old, so, still, young, i, did, the, correct, introduction, by, separating, them, swapping, scents, eating, near, eachother, etc, and, everything, was, going, really, well, they, started, playing, all, the, time, licking, eachother, and, seemed...","[cat, acting, anxious, getting, new, kitten, got, new, kitten, month, ago, introduced, cat, year, old, still, young, correct, introduction, separating, swapping, scents, eating, near, eachother, etc, everything, going, really, well, started, playing, time, licking, eachother, seemed, best, buds, suddenly, older, cat, hissing, time, growling, never, playing, weird, thing, still, catch, napping,..."


In [120]:
dog_data[['full_content_dog_token','full_content_dog_nostop']].tail()

,full_content_dog_token,full_content_dog_nostop
5890,"[dog, has, lost, all, her, stomach, and, chest, hair, my, year, old, french, bulldog, has, lost, all, her, hair, since, three, months, ago, i, don, t, know, if, it, s, due, to, stress, we, got, a, new, puppy, exactly, three, months, ago, she, had, all, her, hair, back, then, or, due, to, a, possible, medical, illness, due, to, age, i, took, her, to, the, vet, and, all, tests, came, out, normal...","[dog, lost, stomach, chest, hair, year, old, french, bulldog, lost, hair, since, three, months, ago, know, due, stress, got, new, puppy, exactly, three, months, ago, hair, back, due, possible, medical, illness, due, age, took, vet, tests, came, normal, asked, xray, look, tumors, dr, knows, ultrasounds, perform, test, worried, put, diet, increased, activity, lost, pounds, comparison, least, yea..."
5891,"[my, toy, poodle, s, fur, has, been, thinning, and, falling, out, in, one, spot, on, her, back, what, should, i, do, ]","[toy, poodle, fur, thinning, falling, one, spot, back, ]"
5892,"[my, dog, wants, to, be, let, out, every, night, in, the, middle, of, the, night, am, he, s, around, years, old, and, he, s, wonderfully, house, trained, like, never, has, an, accident, ever, this, is, a, new, annoying, habit, that, has, started, within, the, past, couple, months, we, always, let, him, out, before, we, go, to, bed, around, pm, and, his, last, meal, is, usually, around, pm, his...","[dog, wants, let, every, night, middle, night, around, years, old, wonderfully, house, trained, like, never, accident, ever, new, annoying, habit, started, within, past, couple, months, always, let, go, bed, around, pm, last, meal, usually, around, pm, water, typically, empty, towards, evening, fill, back, potty, incessantly, whine, anxiously, walk, around, bed, let, recommendations]"
5893,"[dogs, getting, spayed, i, have, some, questions, both, my, dogs, are, getting, spayed, on, wednesday, one, dog, is, about, years, old, and, the, other, is, almost, years, old, i, m, nervous, for, the, procedure, but, i, know, that, it, is, routine, and, better, for, them, in, the, long, term, i, planned, their, spay, to, occur, when, i, ll, be, off, from, work, for, a, week, to, be, able, to,...","[dogs, getting, spayed, questions, dogs, getting, spayed, wednesday, one, dog, years, old, almost, years, old, nervous, procedure, know, routine, better, long, term, planned, spay, occur, work, week, able, care, year, old, black, mouth, cur, year, old, black, lab, neither, health, issues, aware, pretty, active, seem, happy, etc, advice, aftercare, steps, come, house, steps, home, walk, updown,..."
5894,"[pup, chews, everything, on, walks, my, month, old, pup, tries, to, chews, everything, on, walks, pinecones, branches, rocks, grass, etc, any, tips, on, how, to, improve, it, i, ve, googled, a, million, things, and, they, say, one, thing, could, be, boredom, that, s, the, whole, point, of, a, walk, get, some, physical, activity, but, he, won, t, even, make, it, far, on, a, walk, and, just, lay...","[pup, chews, everything, walks, month, old, pup, tries, chews, everything, walks, pinecones, branches, rocks, grass, etc, tips, improve, googled, million, things, say, one, thing, could, boredom, whole, point, walk, get, physical, activity, even, make, far, walk, lays, chews, grass, tried, food, puzzles, play, inside, walk, boredom, still, always]"


#### Stemming

In [121]:
ps = nltk.PorterStemmer()

In [122]:
# return list of all stemmed words for their corresponding words in tokenized_text
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

# apply stemming for cat and dog data
cat_data['full_content_cat_stem'] = cat_data['full_content_cat_nostop'].apply(lambda x: stemming(x))
dog_data['full_content_dog_stem'] = dog_data['full_content_dog_nostop'].apply(lambda x: stemming(x))

In [123]:
cat_data[['full_content_cat_nostop','full_content_cat_stem']].tail()

,full_content_cat_nostop,full_content_cat_stem
5927,"[take, cat, hello, everyone, one, brothers, friend, cant, take, care, another, cat, wish, give, someone, really, want, cat, question, take, cat, even, tho, still, live, parents, garden, backyard, cat, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wondering, bother, cat, change, environment, like, tough, adapt, new, style, life, wait, till, get, flat, ok...","[take, cat, hello, everyon, one, brother, friend, cant, take, care, anoth, cat, wish, give, someon, realli, want, cat, question, take, cat, even, tho, still, live, parent, garden, backyard, cat, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wonder, bother, cat, chang, environ, like, tough, adapt, new, style, life, wait, till, get, flat, ok, take, could,..."
5928,"[got, new, kitten, help, get, along, dog]","[got, new, kitten, help, get, along, dog]"
5929,"[teenager, troubles]","[teenag, troubl]"
5930,"[food, sterilized, cats, kitten, recently, sterilized, months, old, worth, transferring, food, sterilized, cats, still, kitten, worth, continuing, feed, food, kittens, im, feeding, blue, buffalo, healthy, growth, natural, kitten, food]","[food, steril, cat, kitten, recent, steril, month, old, worth, transfer, food, steril, cat, still, kitten, worth, continu, feed, food, kitten, im, feed, blue, buffalo, healthi, growth, natur, kitten, food]"
5931,"[cat, acting, anxious, getting, new, kitten, got, new, kitten, month, ago, introduced, cat, year, old, still, young, correct, introduction, separating, swapping, scents, eating, near, eachother, etc, everything, going, really, well, started, playing, time, licking, eachother, seemed, best, buds, suddenly, older, cat, hissing, time, growling, never, playing, weird, thing, still, catch, napping,...","[cat, act, anxiou, get, new, kitten, got, new, kitten, month, ago, introduc, cat, year, old, still, young, correct, introduct, separ, swap, scent, eat, near, eachoth, etc, everyth, go, realli, well, start, play, time, lick, eachoth, seem, best, bud, suddenli, older, cat, hiss, time, growl, never, play, weird, thing, still, catch, nap, new, kitten, lick, sometim, like, hate, constantli, hiss, g..."


In [124]:
dog_data[['full_content_dog_nostop','full_content_dog_stem']].tail()

,full_content_dog_nostop,full_content_dog_stem
5890,"[dog, lost, stomach, chest, hair, year, old, french, bulldog, lost, hair, since, three, months, ago, know, due, stress, got, new, puppy, exactly, three, months, ago, hair, back, due, possible, medical, illness, due, age, took, vet, tests, came, normal, asked, xray, look, tumors, dr, knows, ultrasounds, perform, test, worried, put, diet, increased, activity, lost, pounds, comparison, least, yea...","[dog, lost, stomach, chest, hair, year, old, french, bulldog, lost, hair, sinc, three, month, ago, know, due, stress, got, new, puppi, exactli, three, month, ago, hair, back, due, possibl, medic, ill, due, age, took, vet, test, came, normal, ask, xray, look, tumor, dr, know, ultrasound, perform, test, worri, put, diet, increas, activ, lost, pound, comparison, least, year, back, weight, rib, wi..."
5891,"[toy, poodle, fur, thinning, falling, one, spot, back, ]","[toy, poodl, fur, thin, fall, one, spot, back, ]"
5892,"[dog, wants, let, every, night, middle, night, around, years, old, wonderfully, house, trained, like, never, accident, ever, new, annoying, habit, started, within, past, couple, months, always, let, go, bed, around, pm, last, meal, usually, around, pm, water, typically, empty, towards, evening, fill, back, potty, incessantly, whine, anxiously, walk, around, bed, let, recommendations]","[dog, want, let, everi, night, middl, night, around, year, old, wonder, hous, train, like, never, accid, ever, new, annoy, habit, start, within, past, coupl, month, alway, let, go, bed, around, pm, last, meal, usual, around, pm, water, typic, empti, toward, even, fill, back, potti, incessantli, whine, anxious, walk, around, bed, let, recommend]"
5893,"[dogs, getting, spayed, questions, dogs, getting, spayed, wednesday, one, dog, years, old, almost, years, old, nervous, procedure, know, routine, better, long, term, planned, spay, occur, work, week, able, care, year, old, black, mouth, cur, year, old, black, lab, neither, health, issues, aware, pretty, active, seem, happy, etc, advice, aftercare, steps, come, house, steps, home, walk, updown,...","[dog, get, spay, question, dog, get, spay, wednesday, one, dog, year, old, almost, year, old, nervou, procedur, know, routin, better, long, term, plan, spay, occur, work, week, abl, care, year, old, black, mouth, cur, year, old, black, lab, neither, health, issu, awar, pretti, activ, seem, happi, etc, advic, aftercar, step, come, hous, step, home, walk, updown, step, go, peepoop, outsid, potti..."
5894,"[pup, chews, everything, walks, month, old, pup, tries, chews, everything, walks, pinecones, branches, rocks, grass, etc, tips, improve, googled, million, things, say, one, thing, could, boredom, whole, point, walk, get, physical, activity, even, make, far, walk, lays, chews, grass, tried, food, puzzles, play, inside, walk, boredom, still, always]","[pup, chew, everyth, walk, month, old, pup, tri, chew, everyth, walk, pinecon, branch, rock, grass, etc, tip, improv, googl, million, thing, say, one, thing, could, boredom, whole, point, walk, get, physic, activ, even, make, far, walk, lay, chew, grass, tri, food, puzzl, play, insid, walk, boredom, still, alway]"


#### Lemmatisation

In [125]:
wn = nltk.WordNetLemmatizer()

In [126]:
# return list of all lemmatized words for their corresponding words in tokenized_text
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

# apply lemmatisation to cat and dog data
cat_data['full_content_cat_lem'] = cat_data['full_content_cat_nostop'].apply(lambda x: lemmatizing(x))
dog_data['full_content_dog_lem'] = dog_data['full_content_dog_nostop'].apply(lambda x: lemmatizing(x))

In [127]:
cat_data[['full_content_cat_nostop','full_content_cat_lem']].tail()

,full_content_cat_nostop,full_content_cat_lem
5927,"[take, cat, hello, everyone, one, brothers, friend, cant, take, care, another, cat, wish, give, someone, really, want, cat, question, take, cat, even, tho, still, live, parents, garden, backyard, cat, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wondering, bother, cat, change, environment, like, tough, adapt, new, style, life, wait, till, get, flat, ok...","[take, cat, hello, everyone, one, brother, friend, cant, take, care, another, cat, wish, give, someone, really, want, cat, question, take, cat, even, tho, still, live, parent, garden, backyard, cat, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wondering, bother, cat, change, environment, like, tough, adapt, new, style, life, wait, till, get, flat, ok, ..."
5928,"[got, new, kitten, help, get, along, dog]","[got, new, kitten, help, get, along, dog]"
5929,"[teenager, troubles]","[teenager, trouble]"
5930,"[food, sterilized, cats, kitten, recently, sterilized, months, old, worth, transferring, food, sterilized, cats, still, kitten, worth, continuing, feed, food, kittens, im, feeding, blue, buffalo, healthy, growth, natural, kitten, food]","[food, sterilized, cat, kitten, recently, sterilized, month, old, worth, transferring, food, sterilized, cat, still, kitten, worth, continuing, feed, food, kitten, im, feeding, blue, buffalo, healthy, growth, natural, kitten, food]"
5931,"[cat, acting, anxious, getting, new, kitten, got, new, kitten, month, ago, introduced, cat, year, old, still, young, correct, introduction, separating, swapping, scents, eating, near, eachother, etc, everything, going, really, well, started, playing, time, licking, eachother, seemed, best, buds, suddenly, older, cat, hissing, time, growling, never, playing, weird, thing, still, catch, napping,...","[cat, acting, anxious, getting, new, kitten, got, new, kitten, month, ago, introduced, cat, year, old, still, young, correct, introduction, separating, swapping, scent, eating, near, eachother, etc, everything, going, really, well, started, playing, time, licking, eachother, seemed, best, bud, suddenly, older, cat, hissing, time, growling, never, playing, weird, thing, still, catch, napping, n..."


In [128]:
dog_data[['full_content_dog_nostop','full_content_dog_lem']].tail()

,full_content_dog_nostop,full_content_dog_lem
5890,"[dog, lost, stomach, chest, hair, year, old, french, bulldog, lost, hair, since, three, months, ago, know, due, stress, got, new, puppy, exactly, three, months, ago, hair, back, due, possible, medical, illness, due, age, took, vet, tests, came, normal, asked, xray, look, tumors, dr, knows, ultrasounds, perform, test, worried, put, diet, increased, activity, lost, pounds, comparison, least, yea...","[dog, lost, stomach, chest, hair, year, old, french, bulldog, lost, hair, since, three, month, ago, know, due, stress, got, new, puppy, exactly, three, month, ago, hair, back, due, possible, medical, illness, due, age, took, vet, test, came, normal, asked, xray, look, tumor, dr, know, ultrasound, perform, test, worried, put, diet, increased, activity, lost, pound, comparison, least, year, back..."
5891,"[toy, poodle, fur, thinning, falling, one, spot, back, ]","[toy, poodle, fur, thinning, falling, one, spot, back, ]"
5892,"[dog, wants, let, every, night, middle, night, around, years, old, wonderfully, house, trained, like, never, accident, ever, new, annoying, habit, started, within, past, couple, months, always, let, go, bed, around, pm, last, meal, usually, around, pm, water, typically, empty, towards, evening, fill, back, potty, incessantly, whine, anxiously, walk, around, bed, let, recommendations]","[dog, want, let, every, night, middle, night, around, year, old, wonderfully, house, trained, like, never, accident, ever, new, annoying, habit, started, within, past, couple, month, always, let, go, bed, around, pm, last, meal, usually, around, pm, water, typically, empty, towards, evening, fill, back, potty, incessantly, whine, anxiously, walk, around, bed, let, recommendation]"
5893,"[dogs, getting, spayed, questions, dogs, getting, spayed, wednesday, one, dog, years, old, almost, years, old, nervous, procedure, know, routine, better, long, term, planned, spay, occur, work, week, able, care, year, old, black, mouth, cur, year, old, black, lab, neither, health, issues, aware, pretty, active, seem, happy, etc, advice, aftercare, steps, come, house, steps, home, walk, updown,...","[dog, getting, spayed, question, dog, getting, spayed, wednesday, one, dog, year, old, almost, year, old, nervous, procedure, know, routine, better, long, term, planned, spay, occur, work, week, able, care, year, old, black, mouth, cur, year, old, black, lab, neither, health, issue, aware, pretty, active, seem, happy, etc, advice, aftercare, step, come, house, step, home, walk, updown, step, g..."
5894,"[pup, chews, everything, walks, month, old, pup, tries, chews, everything, walks, pinecones, branches, rocks, grass, etc, tips, improve, googled, million, things, say, one, thing, could, boredom, whole, point, walk, get, physical, activity, even, make, far, walk, lays, chews, grass, tried, food, puzzles, play, inside, walk, boredom, still, always]","[pup, chew, everything, walk, month, old, pup, try, chew, everything, walk, pinecone, branch, rock, grass, etc, tip, improve, googled, million, thing, say, one, thing, could, boredom, whole, point, walk, get, physical, activity, even, make, far, walk, lay, chew, grass, tried, food, puzzle, play, inside, walk, boredom, still, always]"


#### Remove Common Words for Both r/CatAdvice and r/DogAdvice

Keywords such as 'cat', 'dog' related to the categories of the subreddits were removed as the intention is to build a predition model based on freetext inputed into the chatbot. In addition, the common words found in both subreddits (e.g. vet, veterinarian, month, year) are listed in the set of codes below. After looking at the data, these words were used frequently in both topics. Having them in the dataset for both categories would reduce the differences between them and ultimately reduce the ability of the model to correctly predict the topic based on the word(s) fed into the chatbot.

In [143]:
# Words related to the categories, 'cat' and 'dog', and words that are common in both categories
removal_list = ['cat', 'cats', 'kitten', 'kittens', 'kitty', 'dog', 'dogs', 'pup', 'puppy', 'vet', 'veterinarian', 'month', 'mth', 'year', 'yr', 'old']

# retain and store word in "text" if it is not found in removal_list
def remove_words(tokenized_list):
    text = [word for word in tokenized_list if word not in removal_list]
    return text

#remove stopwords for cat and dog data sets
cat_data['full_content_cat_rm_com_wds'] = cat_data['full_content_cat_lem'].apply(lambda x: remove_words(x))
dog_data['full_content_dog_rm_com_wds'] = dog_data['full_content_dog_lem'].apply(lambda x: remove_words(x))

In [144]:
cat_data[['full_content_cat_lem','full_content_cat_rm_com_wds']].tail()

,full_content_cat_lem,full_content_cat_rm_com_wds
5927,"[take, cat, hello, everyone, one, brother, friend, cant, take, care, another, cat, wish, give, someone, really, want, cat, question, take, cat, even, tho, still, live, parent, garden, backyard, cat, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wondering, bother, cat, change, environment, like, tough, adapt, new, style, life, wait, till, get, flat, ok, ...","[take, hello, everyone, one, brother, friend, cant, take, care, another, wish, give, someone, really, want, question, take, even, tho, still, live, parent, garden, backyard, wander, around, stuff, thing, wait, contract, work, take, flat, garden, regular, flat, wondering, bother, change, environment, like, tough, adapt, new, style, life, wait, till, get, flat, ok, take, could, also, lock, door,..."
5928,"[got, new, kitten, help, get, along, dog]","[got, new, help, get, along]"
5929,"[teenager, trouble]","[teenager, trouble]"
5930,"[food, sterilized, cat, kitten, recently, sterilized, month, old, worth, transferring, food, sterilized, cat, still, kitten, worth, continuing, feed, food, kitten, im, feeding, blue, buffalo, healthy, growth, natural, kitten, food]","[food, sterilized, recently, sterilized, worth, transferring, food, sterilized, still, worth, continuing, feed, food, im, feeding, blue, buffalo, healthy, growth, natural, food]"
5931,"[cat, acting, anxious, getting, new, kitten, got, new, kitten, month, ago, introduced, cat, year, old, still, young, correct, introduction, separating, swapping, scent, eating, near, eachother, etc, everything, going, really, well, started, playing, time, licking, eachother, seemed, best, bud, suddenly, older, cat, hissing, time, growling, never, playing, weird, thing, still, catch, napping, n...","[acting, anxious, getting, new, got, new, ago, introduced, still, young, correct, introduction, separating, swapping, scent, eating, near, eachother, etc, everything, going, really, well, started, playing, time, licking, eachother, seemed, best, bud, suddenly, older, hissing, time, growling, never, playing, weird, thing, still, catch, napping, new, licking, sometimes, like, hate, constantly, h..."


In [145]:
dog_data[['full_content_dog_lem','full_content_dog_rm_com_wds']].tail()

,full_content_dog_lem,full_content_dog_rm_com_wds
5890,"[dog, lost, stomach, chest, hair, year, old, french, bulldog, lost, hair, since, three, month, ago, know, due, stress, got, new, puppy, exactly, three, month, ago, hair, back, due, possible, medical, illness, due, age, took, vet, test, came, normal, asked, xray, look, tumor, dr, know, ultrasound, perform, test, worried, put, diet, increased, activity, lost, pound, comparison, least, year, back...","[lost, stomach, chest, hair, french, bulldog, lost, hair, since, three, ago, know, due, stress, got, new, exactly, three, ago, hair, back, due, possible, medical, illness, due, age, took, test, came, normal, asked, xray, look, tumor, dr, know, ultrasound, perform, test, worried, put, diet, increased, activity, lost, pound, comparison, least, back, weight, rib, wide, sudden, alopecia, plus, wid..."
5891,"[toy, poodle, fur, thinning, falling, one, spot, back, ]","[toy, poodle, fur, thinning, falling, one, spot, back, ]"
5892,"[dog, want, let, every, night, middle, night, around, year, old, wonderfully, house, trained, like, never, accident, ever, new, annoying, habit, started, within, past, couple, month, always, let, go, bed, around, pm, last, meal, usually, around, pm, water, typically, empty, towards, evening, fill, back, potty, incessantly, whine, anxiously, walk, around, bed, let, recommendation]","[want, let, every, night, middle, night, around, wonderfully, house, trained, like, never, accident, ever, new, annoying, habit, started, within, past, couple, always, let, go, bed, around, pm, last, meal, usually, around, pm, water, typically, empty, towards, evening, fill, back, potty, incessantly, whine, anxiously, walk, around, bed, let, recommendation]"
5893,"[dog, getting, spayed, question, dog, getting, spayed, wednesday, one, dog, year, old, almost, year, old, nervous, procedure, know, routine, better, long, term, planned, spay, occur, work, week, able, care, year, old, black, mouth, cur, year, old, black, lab, neither, health, issue, aware, pretty, active, seem, happy, etc, advice, aftercare, step, come, house, step, home, walk, updown, step, g...","[getting, spayed, question, getting, spayed, wednesday, one, almost, nervous, procedure, know, routine, better, long, term, planned, spay, occur, work, week, able, care, black, mouth, cur, black, lab, neither, health, issue, aware, pretty, active, seem, happy, etc, advice, aftercare, step, come, house, step, home, walk, updown, step, go, peepoop, outside, potty, pad, first, day, get, car, proc..."
5894,"[pup, chew, everything, walk, month, old, pup, try, chew, everything, walk, pinecone, branch, rock, grass, etc, tip, improve, googled, million, thing, say, one, thing, could, boredom, whole, point, walk, get, physical, activity, even, make, far, walk, lay, chew, grass, tried, food, puzzle, play, inside, walk, boredom, still, always]","[chew, everything, walk, try, chew, everything, walk, pinecone, branch, rock, grass, etc, tip, improve, googled, million, thing, say, one, thing, could, boredom, whole, point, walk, get, physical, activity, even, make, far, walk, lay, chew, grass, tried, food, puzzle, play, inside, walk, boredom, still, always]"


#### Labelling of Data & Appending Cat and Dog Dataframes

In [146]:
# create a new data frame for each stage of text processing
# original text = title + selftext(if any)
df_cat = pd.DataFrame().assign(original_text=cat_data['full_content_cat'], removed_punc=cat_data['full_content_cat_punc'], 
                                    tokenized=cat_data['full_content_cat_token'],  removed_stop=cat_data['full_content_cat_nostop'],
                                    stem=cat_data['full_content_cat_stem'], lem=cat_data['full_content_cat_lem'], rm_com_words=cat_data['full_content_cat_rm_com_wds'])

df_dog = pd.DataFrame().assign(original_text=dog_data['full_content_dog'], removed_punc=dog_data['full_content_dog_punc'], 
                                    tokenized=dog_data['full_content_dog_token'],  removed_stop=dog_data['full_content_dog_nostop'],
                                    stem=dog_data['full_content_dog_stem'], lem=dog_data['full_content_dog_lem'], rm_com_words=dog_data['full_content_dog_rm_com_wds'])

In [147]:
# labelling of data for 'cat?': cat == 1; dog == 0
df_cat['cat?'] = None
df_cat.loc[:,'cat?'] = 1

df_dog['cat?'] = None
df_dog.loc[:,'cat?'] = 0

In [148]:
#append df_cat and df_dog to create a combined dataframe: full_df
full_df = df_cat.append(df_dog)
full_df.reset_index(drop=True, inplace=True)
full_df

C:\Users\gyinh\AppData\Local\Temp\ipykernel_14660\908103533.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = df_cat.append(df_dog)


,original_text,removed_punc,tokenized,removed_stop,stem,lem,rm_com_words,cat?
0,"Transporting cats on a . hr drive to move to a new place??? | Best way to make them comfortable? I have cats and carriers but I feel the carriers are a little small for such a long drive. It's fine when they go to the vet who is close. One cat is lbs the other is , we have the largest cat carriers we could find. Do they need to potty or have food/water for a long drive? Or will they be too s...",Transporting cats on a hr drive to move to a new place Best way to make them comfortable I have cats and carriers but I feel the carriers are a little small for such a long drive Its fine when they go to the vet who is close One cat is lbs the other is we have the largest cat carriers we could find Do they need to potty or have foodwater for a long drive Or will they be too stressed to do...,"[transporting, cats, on, a, hr, drive, to, move, to, a, new, place, best, way, to, make, them, comfortable, i, have, cats, and, carriers, but, i, feel, the, carriers, are, a, little, small, for, such, a, long, drive, its, fine, when, they, go, to, the, vet, who, is, close, one, cat, is, lbs, the, other, is, we, have, the, largest, cat, carriers, we, could, find, do, they, need, to, potty, or, ...","[transporting, cats, hr, drive, move, new, place, best, way, make, comfortable, cats, carriers, feel, carriers, little, small, long, drive, fine, go, vet, close, one, cat, lbs, largest, cat, carriers, could, find, need, potty, foodwater, long, drive, stressed, large, cat, scream, loud, hrs, straight, breaks, record, thus, far, starts, screaming, soon, gets, carrier, closed, people, car, especi...","[transport, cat, hr, drive, move, new, place, best, way, make, comfort, cat, carrier, feel, carrier, littl, small, long, drive, fine, go, vet, close, one, cat, lb, largest, cat, carrier, could, find, need, potti, foodwat, long, drive, stress, larg, cat, scream, loud, hr, straight, break, record, thu, far, start, scream, soon, get, carrier, close, peopl, car, especi, driver, keep, sane, calm, s...","[transporting, cat, hr, drive, move, new, place, best, way, make, comfortable, cat, carrier, feel, carrier, little, small, long, drive, fine, go, vet, close, one, cat, lb, largest, cat, carrier, could, find, need, potty, foodwater, long, drive, stressed, large, cat, scream, loud, hr, straight, break, record, thus, far, start, screaming, soon, get, carrier, closed, people, car, especially, driv...","[transporting, hr, drive, move, new, place, best, way, make, comfortable, carrier, feel, carrier, little, small, long, drive, fine, go, close, one, lb, largest, carrier, could, find, need, potty, foodwater, long, drive, stressed, large, scream, loud, hr, straight, break, record, thus, far, start, screaming, soon, get, carrier, closed, people, car, especially, driver, keep, sane, calm, screamin...",1
1,My cat wont eat. Vet determined he is not sick. Desperate for advice.,My cat wont eat Vet determined he is not sick Desperate for advice,"[my, cat, wont, eat, vet, determined, he, is, not, sick, desperate, for, advice]","[cat, wont, eat, vet, determined, sick, desperate, advice]","[cat, wont, eat, vet, determin, sick, desper, advic]","[cat, wont, eat, vet, determined, sick, desperate, advice]","[wont, eat, determined, sick, desperate, advice]",1
2,"Two month old outdoor kittens, one injured one missing. Can the injured one help me find the missing one?",Two month old outdoor kittens one injured one missing Can the injured one help me find the missing one,"[two, month, old, outdoor, kittens, one, injured, one, missing, can, the, injured, one, help, me, find, the, missing, one]","[two, month, old, outdoor, kittens, one, injured, one, missing, injured, one, help, find, missing, one]","[two, month, old, outdoor, kitten, one, injur, one, miss, injur, one, help, find, miss, one]","[two, month, old, outdoor, kitten, one, injured, one, missing, injured, one, help, find, missing, one]","[two, outdoor, o

In [ ]:
# save processed dataframe as full_df
full_df.to_csv('../data/full_df_rm_com_wds.csv')